Import the pandas, numpy and plotly.express libraries.
Read data from the csv file and create a data frame.

In [174]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:20,.2f}'.format
file = r'Largest US Venture Funding Deals Of 2023.csv'
df = pd.read_csv(file)
df.head()

,Company,Amount,Lead investors,Valuation,Industry,Date reported
0,OpenAI,"$10,000,000,000",Microsoft,NaN,Artificial intelligence,1/23/23
1,Stripe,"$6,500,000,000",NaN,"$50,000,000,000",Fintech,3/15/23
2,Inflection AI,"$1,300,000,000","Microsoft, Reid Hoffman, Bill Gates, Eric Schm...","$4,000,000,000",Artificial intelligence,6/29/23
3,Anthropic,"$1,250,000,000",Amazon,"$4,000,000,000",Artificial intelligence,9/25/23
4,Generate Capital,"$1,030,900,000",NaN,NaN,Energy,1/6/23


Print a concise summary of the data frame.
There are 171 rows and 6 columns in total.
The columns Company, Amount, Industry and Date Reported have no null values. Lead investors has 26 null values and Valuation has
124 null values.

In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Company         171 non-null    object
 1   Amount          171 non-null    object
 2   Lead investors  145 non-null    object
 3   Valuation       47 non-null     object
 4   Industry        171 non-null    object
 5   Date reported   171 non-null    object
dtypes: object(6)
memory usage: 8.1+ KB


Check the number of distinct values in each column.

In [176]:
df.nunique()

Company           161
Amount             69
Lead investors    133
Valuation          29
Industry           63
Date reported      97
dtype: int64

Preprocessing the data.
Replace an "na" value in Valuation with empty.
Replace empty Valuation with \\$0.
Strip \\$ from Amount and Valuation and convert to integer.
Format the Date reported column as M/D/Y.
Create a Month column from Date Reported.
Create a Month Name column from Date Reported.
Replace missing values in Lead investors with "Other".

In [177]:
df['Valuation'].replace('na', float('nan'), inplace=True)
df['Valuation'].fillna('$0', inplace=True)
df['Amount'] = df['Amount'].str.replace('[$,]', '', regex=True).astype(np.int64)
df['Valuation'] = df['Valuation'].str.replace('[$,]', '', regex=True).astype(np.int64)
df['Date reported'] = pd.to_datetime(df['Date reported'], format='%m/%d/%y')
df['Month'] = df['Date reported'].dt.month
df['Month Name'] = df['Date reported'].dt.month_name()
df['Lead investors'].fillna('Other', inplace=True)
df.head()

,Company,Amount,Lead investors,Valuation,Industry,Date reported,Month,Month Name
0,OpenAI,10000000000,Microsoft,0,Artificial intelligence,2023-01-23,1,January
1,Stripe,6500000000,Other,50000000000,Fintech,2023-03-15,3,March
2,Inflection AI,1300000000,"Microsoft, Reid Hoffman, Bill Gates, Eric Schm...",4000000000,Artificial intelligence,2023-06-29,6,June
3,Anthropic,1250000000,Amazon,4000000000,Artificial intelligence,2023-09-25,9,September
4,Generate Capital,1030900000,Other,0,Energy,2023-01-06,1,January


General descriptive statistics of numerical columns.
The average funding amount is \\$300 million while the average valuation of a company is \\$1.1 billion. The last month included in the dataset is September.

In [178]:
numerical_features = df.select_dtypes(include=[np.number])
numerical_features.describe()

,Amount,Valuation,Month
count,171.00,171.00,171.00
mean,"301,257,894.74","1,160,964,912.28",5.24
std,"907,398,912.14","5,155,941,784.11",2.72
min,"100,000,000.00",0.00,1.00
25%,"104,500,000.00",0.00,3.00
50%,"150,000,000.00",0.00,5.00
75%,"232,500,000.00","612,500,000.00",8.00
max,"10,000,000,000.00","50,000,000,000.00",9.00


Get a list of unique values for the Industry column.
At a glance, there is at least one mispelled version of a valid industry name.

In [179]:
set(df['Industry'].unique())

{'Accounting',
 'Advertising',
 'Aerospace',
 'Agriculture',
 'Analytics',
 'Apparel',
 'Artifical intelligence',
 'Artificial intelligence',
 'Auto insurance',
 'Automotive',
 'Autonomous vehicles',
 'Batteries',
 'Big data',
 'Biotech',
 'Cannabis',
 'Cleantech',
 'Cloud',
 'Compliance',
 'Construction',
 'Cryptocurrency',
 'Cybersecurity',
 'Data',
 'Database',
 'Digital media',
 'Drones',
 'E-commerce',
 'Energy',
 'Enterprise software',
 'Enviromental consulting',
 'Financial services',
 'Fintech',
 'Fitness',
 'Fleet Management',
 'Hardware',
 'Health care',
 'Health diagnostic',
 'Healthc are',
 'Human resources',
 'Identity management',
 'Insurance',
 'Logistics',
 'Manufacturing',
 'Marketing',
 'Media',
 'Mining',
 'Network software',
 'Networking',
 'Neuroscience',
 'Procurement',
 'Property management',
 'Real estate',
 'Renewable energy',
 'SaaS',
 'Sales',
 'Security',
 'Semiconductor',
 'Space',
 'Space tech',
 'Sports',
 'Sustainability',
 'Telecommunications',
 'Transp

Correct the spelling mistakes in the Industry column.
There are three industries affected: Artificial intelligence, Healthcare and Space

In [180]:
def change(x):
    if 'Artif' in x:
        return 'Artificial Intelligence'
    elif 'Healthc are' in x:
        return 'Healthcare'
    elif 'Health care' in x:
        return 'Healthcare'
    elif 'Space tech' in x:
        return 'Space'
    else:
        return x
df['Industry'] = df['Industry'].map(change)

Create a vertical bar chart showing total funding by month.
The highest total was in January, while the lowest was in July. There was a slowdown during the summer period, but it seems to be recovering in September, which is the last month in the data.

In [181]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
df['Month Name'] = pd.Categorical(df['Month Name'], categories=month_order, ordered=True)
df = df.sort_values('Month Name')
px.bar(data_frame=df[['Month Name', 'Amount']].groupby(by='Month Name').sum().reset_index(), x='Month Name', y='Amount',
      labels={'Amount': 'Total Amount', 'Month Name': 'Month'},
      title = 'Total Funding by Month in 2023')

Keep the largest 9 Industries by funding amount and group all the remaining ones in an "Other" category.

In [182]:
industry_totals = df.groupby('Industry')['Amount'].sum().reset_index()
top_industries = industry_totals.nlargest(9, 'Amount')['Industry'].tolist()
df['Industry'] = df['Industry'].apply(lambda x: x if x in top_industries else 'Other')

Create a horizontal bar chart showing total funding by lead investor further broken down by industry.
Microsoft is by far the largest individual contributor to total funding in 2023 with a focus exclusively on Artificial Intelligence. Fintech is has the most investors, although the total amount is less than Microsoft alone.

In [183]:
grouped_df = df.groupby(['Lead investors', 'Industry']).agg({'Amount': 'sum'}).reset_index()

top_investors = grouped_df.groupby('Lead investors')['Amount'].sum().sort_values(ascending=False).head(10).index
filtered_df = grouped_df[grouped_df['Lead investors'].isin(top_investors)]
fig = px.bar(filtered_df, x='Amount', y='Lead investors', color='Industry',
             title='Total Funding by Lead Investor and Industry',
             labels={'Amount': 'Total Amount'},
             category_orders={"Lead investors": filtered_df.sort_values(by='Amount', ascending=False)['Lead investors'].tolist()})
fig.update_yaxes(ticktext=[i.split()[0] for i in filtered_df['Lead investors']], tickvals=filtered_df['Lead investors'])
fig.show()

Create a donut chart showing total funding by Industry.
Artificial Intelligence has the highest slice at 31.8%, followed by Fintech and Biotech. Every other Industry has less than 4%
of the total each.

In [184]:
grouped_df = df.groupby('Industry').agg({'Amount': 'sum'}).reset_index()
top_industries = grouped_df.nlargest(10, 'Amount')
fig = px.pie(top_industries, values='Amount', names='Industry',
             title='Total Funding by Industry',
             hole=0.5,  
             labels={'Amount': 'Total Amount'})
fig.show()

Check the funding amount for the top ten companies by valuation.
The number one company is Stripe, valued at \\$50 bliion, which received a total of \\$6.5 billion in funding in 2023.
The best ratio of funding to valuation belongs to Anthropic, valued at \\$8.1 billion, which received \\$2.1 billion in 2023.

In [185]:
grouped_df = df.groupby('Company').agg({'Amount': 'sum', 'Valuation': 'sum'}).reset_index()
top_companies = grouped_df.nlargest(10, 'Valuation')
print(top_companies.to_string(index=False))

     Company     Amount   Valuation
      Stripe 6500000000 50000000000
  Databricks  500000000 43000000000
         Wiz  300000000 10000000000
   Anthropic 2100000000  8100000000
        Ramp  300000000  5800000000
Sierra Space  290000000  5300000000
Hugging Face  235000000  4500000000
    OneTrust  150000000  4500000000
  AlphaSense  250000000  4300000000
     Zipline  330000000  4200000000
